# Diagnostics vent géostrophique, divergence et tourbillon

Auteur : FERRY Frédéric (DESR/ENM/C3M) - novembre 2021

On utilise ici les données quotidiennes NCEP de géopotentiel, vent zonal, vent méridien et vitesse verticale pour l'année 2016. Pour récupérer les données d'autres années aller à l'adresse suivante : ftp://ftp2.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/

Concepts illustrés :

- Tracé de champs de géopotentiel et de vent horizontal
- Diagnostic du vent géostrophique à partir du champ de géopotentiel
- Calcul du du vent agéostrophique et lien avec les vitesses verticales de grande échelle
- Calcul de la divergence du vent
- Calcul de la composante verticale du tourbillon
- Tracé de cartes (cartopy)

<div class="alert alert-warning">
<b>En jaune : </b> code à compléter
</div>

In [ ]:
import os

import numpy as np

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

import shapely.geometry as sgeom
from area import area

import warnings
warnings.filterwarnings('ignore')

In [ ]:
diri="./data/"
dir_figs="./figs/"
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)

In [ ]:
re=6.37e6
g=9.81
omega=7.292e-5

# Ouverture des données

In [ ]:
year='2016'
month='01'
day='15'
date=year+'-'+month+'-'+day

zlev=300
ulev=300
wlev=600

In [ ]:
#--  Read z, t, u, v, w data for specified vertical level
fz    = xr.open_dataset(diri+'hgt.'+year+'.nc').sel(level=zlev).sel(time=date).squeeze()
fu    = xr.open_dataset(diri+'uwnd.'+year+'.nc').sel(level=ulev).sel(time=date).squeeze()
fv    = xr.open_dataset(diri+'vwnd.'+year+'.nc').sel(level=ulev).sel(time=date).squeeze()
fw    = xr.open_dataset(diri+'omega.'+year+'.nc').sel(level=wlev).sel(time=date).squeeze()

print(fz)
print(fu)
print(fv)
print(fw)

#--  Extract variables
z0 = fz['hgt']
u0 = fu['uwnd']
v0 = fv['vwnd']
w0 = fw['omega']
print(z0.shape)

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
z1 = lonflip(z0)
u1 = lonflip(u0)
v1 = lonflip(v0)
w1 = lonflip(w0)

lon0  = z1.lon.values

In [ ]:
latS = 30
latN = 70
lonW = -90
lonE = 20

z=z1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))
u=u1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))
v=v1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))
w=w1.sel(lat=slice(latN,latS)).sel(lon=slice(lonW,lonE))

print(z.shape)

lon  = z.lon.values
lat  = z.lat.values

print('Latitudes : ', lat)
print('Longitudes : ', lon)

# Diagnostics

In [ ]:
xlon,ylat=np.meshgrid(lon,lat)

print(xlon.shape) # lon x lat
print(ylat.shape) # lon x lat

In [ ]:
dlony,dlonx=np.gradient(xlon)
dlaty,dlatx=np.gradient(ylat)

#print(dlonx)
#print(dlaty)

$$dx = a cos(\phi)d\lambda$$
$$dy = a d\phi$$

In [ ]:
dx=re*np.cos(ylat*np.pi/180)*dlonx*np.pi/180
dy=re*dlaty*np.pi/180

<div class="alert alert-warning">
<b>Complétez le code avec le calcul du paramètre de Coriolis (f) : </b>
$$f = 2\Omega sin(\phi)$$
</div>

In [ ]:
f=

<div class="alert alert-warning">
<b>Complétez le code avec le calcul du vent géostrophique (ug, vg) : </b>
$$u_g = -\frac{g}{f} \frac{\partial z}{\partial y}$$
$$v_g = \frac{g}{f} \frac{\partial z}{\partial x}$$
</div>

<div class="alert alert-warning">
<b>Complétez le code avec le calcul du vent agéostrophique (ua, va) : </b>
$$u_a = u - u_g$$
$$v_a = v - v_g$$
</div>

In [ ]:
# calculate derivatives
dzdx=np.gradient(z,axis=1)/dx
dzdy=np.gradient(z,axis=0)/dy

# calculate geostrophic wind
ug=
vg=

# calculate ageostrophic wind
ua=
va=

<div class="alert alert-warning">
<b>Complétez le code avec le calcul des modules des vents (horizontal, géostrophique, agéostrophique). </b>
</div>

In [ ]:
hwind_mod=
gwind_mod=
awind_mod=

<div class="alert alert-warning">
<b>Complétez le code avec le calcul de la divergence horizontale (div) et de la divergence géostrophique (divg) :</b>
$$div(\vec{V}_h) = \frac{\partial u}{\partial x}+\frac{\partial v}{\partial y}-\frac{v}{a}tan(\phi)$$
$$div(\vec{V}_g) = \frac{\partial u_g}{\partial x}+\frac{\partial v_g}{\partial y}-\frac{v_g}{a}tan(\phi)$$
</div>

<div class="alert alert-warning">
<b>Complétez le code avec le calcul du tourbillon relatif (vort) et du tourbillon géostrophique (vortg) :</b>
$$\xi = \frac{\partial v}{\partial x}-\frac{\partial u}{\partial y}+\frac{u}{a}tan(\phi)$$
$$\xi_g = \frac{\partial v_g}{\partial x}-\frac{\partial u_g}{\partial y}+\frac{u_g}{a}tan(\phi)$$
</div>

<div class="alert alert-warning">
<b>Complétez le code avec le calcul de la divergence agéostrophique (diva) et du tourbillon agéostrophique (vorta) : </b>
$$div(\vec{V}_a) = \frac{\partial u_a}{\partial x}+\frac{\partial v_a}{\partial y}-\frac{v_a}{a}tan(\phi)$$
$$\xi_a = \frac{\partial v_a}{\partial x}-\frac{\partial u_a}{\partial y}+\frac{u_a}{a}tan(\phi)$$
</div>

In [ ]:
# calculate derivatives
dudx=np.gradient(u,axis=1)/dx
dvdx=np.gradient(v,axis=1)/dx
dudy=np.gradient(u,axis=0)/dy
dvdy=np.gradient(v,axis=0)/dy

dugdx=np.gradient(ug,axis=1)/dx
dvgdx=np.gradient(vg,axis=1)/dx
dugdy=np.gradient(ug,axis=0)/dy
dvgdy=np.gradient(vg,axis=0)/dy

duadx=np.gradient(ua,axis=1)/dx
dvadx=np.gradient(va,axis=1)/dx
duady=np.gradient(ua,axis=0)/dy
dvady=np.gradient(va,axis=0)/dy

# Divergence and vorticity
div=
divg=
diva=

vort=
vortg=
vorta=

# Tracés de cartes

In [ ]:
def plot_background(ax):
    ax.coastlines()
    ax.set_xticks(np.linspace(-180, 180, 19), crs=projection)
    ax.set_yticks(np.linspace(-90, 90, 19), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return ax

In [ ]:
levels_z = np.arange(int(z.min()),int(z.max()),100)
levels_w = np.arange(-0.5,0.6,0.1)
levels_wind = np.arange(30,72.5,2.5)
levels_div = np.arange(-20,21,1)
levels_vor = np.arange(-10,11,1)
levels_f = np.arange(0,2.1,0.1)

wind_slice = slice(None, None, 2)

#projection = ccrs.Orthographic(central_longitude=0, central_latitude=45)
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

In [ ]:
fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Coriolis parameter', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, f*10**4, levels_f, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-4}$ s$^{-1}$')
c = ax.contour(lon, lat, f*10**4, levels_f, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1f', fontsize=10)

figname=dir_figs+'Coriolis_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(12., 12.))
fig.suptitle('Winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(2, 1, 1, projection=projection)
ax.set_title('Horizontal wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
cf = ax.contourf(lon, lat, hwind_mod, levels_wind, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('m/s')

wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(2, 1, 2, projection=projection)
ax.set_title('Streamlines', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

c2= ax.streamplot(lon, lat, np.array(u), np.array(v), transform=ccrs.PlateCarree(),
                  linewidth=1, density=2, color=np.array(hwind_mod))

figname=dir_figs+'wind_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(12., 12.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(2, 1, 1, projection=projection)
ax.set_title('Geopotential height and horizontal wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(2, 1, 2, projection=projection)
ax.set_title('Geopotential height and streamlines', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
c2= ax.streamplot(lon, lat, np.array(u), np.array(v), transform=ccrs.PlateCarree(),
                  linewidth=1, density=2, color=np.array(hwind_mod))

figname=dir_figs+'Z_wind_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(3, 1, 1, projection=projection)
ax.set_title('Geopotential height and horizontal wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(3, 1, 2, projection=projection)
ax.set_title('Geopotential height and geostrophic wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(3, 1, 3, projection=projection)
ax.set_title('Geopotential height and ageostrophic wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ua[wind_slice, wind_slice], va[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

figname=dir_figs+'Z_vh_vg_va_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height and winds at '+str(ulev)+' hPa : '+date, fontsize=16)

ax = fig.add_subplot(4, 1, 1, projection=projection)
ax.set_title('Geopotential height and horizontal wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cf = ax.contourf(lon, lat, hwind_mod, levels_wind, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
cb = fig.colorbar(cf, orientation='vertical')
cb.set_label('m/s')
                  
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(4, 1, 2, projection=projection)
ax.set_title('Geopotential height and geostrophic wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cf = ax.contourf(lon, lat, gwind_mod, levels_wind, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
cb = fig.colorbar(cf, orientation='vertical')
cb.set_label('m/s')

wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(4, 1, 3, projection=projection)
ax.set_title('Geopotential height and ageostrophic wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
#cf = ax.contourf(lon, lat, awind_mod, levels_wind, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
cb = fig.colorbar(cf, orientation='vertical')
cb.set_label('m/s')
                  
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ua[wind_slice, wind_slice], va[wind_slice, wind_slice],
                color='red', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

ax = fig.add_subplot(4, 1, 4, projection=projection)
ax.set_title('Geopotential height at '+str(ulev)+ 'hPa and vertical motion at '+str(wlev)+ 'hPa', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
                
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

cf = ax.contourf(lon, lat, w, levels_w, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='vertical')
cb.set_label('Pa/s')

figname=dir_figs+'Z_vh_vg_va_w_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height and divergence at '+str(ulev)+' hPa : '+date, fontsize=16)
ax = fig.add_subplot(3, 1, 1, projection=projection)
ax.set_title('Horizontal divergence', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, div*10**6, levels_div, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-6}$ s$^{-1}$')
                  
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

ax = fig.add_subplot(3, 1, 2, projection=projection)
ax.set_title('Geostrophic divergence', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, divg*10**6, levels_div, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-6}$ s$^{-1}$')

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

ax = fig.add_subplot(3, 1, 3, projection=projection)
ax.set_title('Ageostrophic divergence', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, diva*10**6, levels_div, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-6}$ s$^{-1}$')
   
c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

figname=dir_figs+'Z_div_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10., 17.))
fig.suptitle('Geopotential height winds and vorticity at '+str(ulev)+' hPa', fontsize=16)
ax = fig.add_subplot(3, 1, 1, projection=projection)
ax.set_title('Geopotential height, relative vorticity and horizontal winds', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, vort*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

ax = fig.add_subplot(3, 1, 2, projection=projection)
ax.set_title('Geopotential height, geostrophic vorticity and geostrophic wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, vortg*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ug[wind_slice, wind_slice], vg[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

ax = fig.add_subplot(3, 1, 3, projection=projection)
ax.set_title('Geopotential height, ageostrophic vorticity and ageostrophic wind', loc='center', fontsize=10)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, vorta*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                ua[wind_slice, wind_slice], va[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

figname=dir_figs+'Z_vort_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>Choisir une zone de 2.5x2.5° dans une région de tourbillon relatif cyclonique. </b>
</div>

In [ ]:
lon1=int(input("Longitude W : "))
lon2=int(input("Longitude E : "))
lat1=int(input("Latitude S : "))
lat2=int(input("Latitude N : "))

In [ ]:
fig = plt.figure(figsize=(15., 8.))
fig.suptitle('Geopotential height winds and vorticity at '+str(ulev)+' hPa', fontsize=16)
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height, relative vorticity and horizontal winds', loc='center', fontsize=10)
plot_background(ax)
box = sgeom.box(minx=lon1, maxx=lon2, miny=lat1, maxy=lat2)
ax.add_geometries([box], projection, facecolor='none', edgecolor='red', linewidths=3)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, vort*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('10$^{-5}$ s$^{-1}$')
wind = ax.barbs(lon[wind_slice], lat[wind_slice],
                u[wind_slice, wind_slice], v[wind_slice, wind_slice],
                color='black', length=6,sizes=dict(emptybarb=0.25, spacing=.2, height=0.5),
                zorder = 10, linewidth=0.95, transform= ccrs.PlateCarree())

c = ax.contour(lon, lat, z, levels_z, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)

figname=dir_figs+'Z_vort_area_'+str(zlev)+'_'+date
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

<div class="alert alert-warning">
<b>La fonction area calcule ici la surface (en m²) incluse dans la zone choisie. Connaissant le lien entre circulation et tourbillon relatif moyen, calculer la circulation autour du circuit. </b>
</div>

In [ ]:
obj = {'type':'Polygon','coordinates':[[[lon1,lat1],[lon1,lat2],[lon2,lat2],[lon2,lat1],[lon1,lat1]]]}
surface=area(obj)
print('Surface incluse dans la zone choisie (m²) : ', surface)
vort_mean=
print('Tourbillon relatif moyen dans la zone choisie (s⁻¹) : ', np.array("%e"%vort_mean))
circulation=
print('Circulation associée (m²s⁻¹,) : ', "%e"%circulation)